# 迁移命令
- 1.makemigrations：将模型生成迁移脚本，模型所在的APP必须放在settings.py中的INSTALLED_APPS中，此命令有以下几个常用选项
    - app_label:后面可以跟一个或者多个APP，那么就只会对这几个APP生成迁移脚本，如果没有任何的app_label，那么会检查INSTALLED_APPS中所有的APP下的模型，针对每一个APP都生成相应的迁移脚本
    - --name:给这个迁移脚本指定一个名字
    - --empty：生成一个空的迁移脚本，如果你想写自己的脚本，可以使用这个命令来实现一个空的文件，然后再在文件汇总写迁移脚本
- 2.migrate：将新的生成的迁移脚本，映射到数据库中，创建新的表或者修改表的结构，以下一些常用的选项
    - app_label：将某个APP下的迁移脚本映射到数据库中，如果没有指定，那么会将所有在INSTALLED_APPS中的APP下的模型都映射到数据库中
    - app_label migrationname:将某个APP下指定名字的migrations文件映射到数据库中
    - --fake：可以将指定的迁移脚本名字添加到数据库中，但是并不会把迁移脚本转换为SQL语句来修改数据库中的表,比如：python manage.py migrate app名字 --fake
    - --fake-initial：将第一次生成的迁移文件版本号记录在数据库中，但并不会真正的执行迁移脚本
- 3.showmigrations：查看某个APP下的迁移文件，如果后面没有APP，那么将查看INSTALLED_APPS中所有的迁移文件
- 4.sqlmigrate：查看某个迁移文件在映射到数据库中的时候，转换的SQL语句
- 关于迁移脚本更多信息，查看文档：https://docs.djangoproject.com/en/3.0/topic/migrations

## migrate
- 工作原理：
    - 1.将相关的迁移脚本翻译成SQL语句，在数据库中执行这个SQL语句
    - 2.如果这个SQL语句执行没有问题，那么就会将这个迁移脚本的名字记录到“django_migrations”中
- migrate如何判断哪些迁移脚本需要执行
    - 它会将代码中的迁移脚本和数据库中的django_migrations中的迁移脚本进行对比，如果发现数据库中，没有这个迁移脚本，那么就会执行这个迁移脚本

# 执行migrate命令的时候报错的解决办法
## 原因
- 执行migrate命令会报错的原因是，数据库中的django_migrations表中的迁移版本记录和代码中的迁移脚本不一致导致的

## 解决办法
- 第一种：使用--fake参数
    - 首先对比数据库中的迁移脚本和代码中的迁移脚本，然后找到哪个不同，之后再使用--fake将代码中的迁移脚本添加到diango_migrations中，但是并不会执行SQL语句，这样就可以避免每次执行migrate的时候，都执行一些重复的迁移脚本
- 第二种：究极解决办法
    - 如果代码中的迁移脚本和数据库中的迁移脚本实在太多，已经搞不清了，那么这时候就可以像这样做：
        - 1.将之前的迁移脚本都删除，重新来过，同时要保证出问题的APP下的所有模型字段名和数据库中的一致
        - 2.将出问题的APP下的所有迁移脚本文件都删掉，再在django_migrations表中将出问题的APP相关的迁移记录都删掉
        - 3.使用makemigrations，重新生成一个迁移脚本
        - 4.使用migrate --fake initial参数，将刚刚生成的迁移脚本，标记为已完成（因为这些模型对应的表，其实都已经在数据库中存在了，不需要再映射了）。然后就可以做其他的迁移了